In [1]:
from proton_driver import client
import time
import plotly.graph_objs as go
from IPython.display import clear_output, display
import pandas as pd
from dash import Dash, html, dcc, Output, Input
#c = client.Client(host='192.168.1.10', port=8463)


In [2]:
num = 1
time_sleep = 3

In [3]:
# def fetch_latest_row():
#     # 每次新建连接，避免连接失效
#     c = client.Client(host='192.168.1.10', port=8463)
#     query = """
#     SELECT 
#         TIMESTAMP, SYMBOL, num_symbols, num_upper_limit, num_lower_limit, total_volume, active_buy_ratio,
#         price_range_0, price_range_1, price_range_2, price_range_3, price_range_4,
#         price_range_5, price_range_6, price_range_7, price_range_8
#     FROM PriceVol_UpDownDist
#     ORDER BY TIMESTAMP ASC LIMIT 10
#     """
#     try:
#         rows = list(c.execute_iter(query))
#         columns = [
#             "TIMESTAMP", "SYMBOL", "num_symbols", "num_upper_limit", "num_lower_limit", "total_volume", "active_buy_ratio",
#             "price_range_0", "price_range_1", "price_range_2", "price_range_3", "price_range_4",
#             "price_range_5", "price_range_6", "price_range_7", "price_range_8"
#         ]
#         df = pd.DataFrame(rows, columns=columns)
#         return df
#     except Exception as e:
#         print("数据库查询异常：", e)
#         return pd.DataFrame()
#     finally:
#         try:
#             c.disconnect()
#         except Exception:
#             pass
# # 转为 DataFrame
# df = fetch_latest_row()

# # 查看前几行
# print(df.head())

In [4]:
# 涨幅区间标签
price_range_labels = [
    "(-20%,-10%]", "(-10%,-5%]", "(-5%,-3%]", "(-3%,0%]", "(0%,0%]", 
    "(0%,3%]", "(3%,5%]", "(5%,10%]", "(10%,20%]"
]

def fetch_latest_row():
    # 每次新建连接，避免连接失效
    c = client.Client(host='192.168.1.10', port=8463)
    query = """
    SELECT 
        TIMESTAMP, SYMBOL, num_symbols, num_upper_limit, num_lower_limit, total_volume, active_buy_ratio,
        price_range_0, price_range_1, price_range_2, price_range_3, price_range_4,
        price_range_5, price_range_6, price_range_7, price_range_8
    FROM PriceVol_UpDownDist
    ORDER BY TIMESTAMP DESC LIMIT 1
    """
    try:
        rows = list(c.execute_iter(query))
        columns = [
            "TIMESTAMP", "SYMBOL", "num_symbols", "num_upper_limit", "num_lower_limit", "total_volume", "active_buy_ratio",
            "price_range_0", "price_range_1", "price_range_2", "price_range_3", "price_range_4",
            "price_range_5", "price_range_6", "price_range_7", "price_range_8"
        ]
        df = pd.DataFrame(rows, columns=columns)
        return df
    except Exception as e:
        print("数据库查询异常：", e)
        return pd.DataFrame()
    finally:
        try:
            c.disconnect()
        except Exception:
            pass

app = Dash()

app.layout = html.Div([
    html.H2('全市场股票涨跌分布', style={'textAlign': 'center','color': 'white'}),
    html.Div(id='summary-box', style={'display': 'flex', 'justifyContent': 'center', 'gap': '40px', 'fontSize': '18px', 'color': 'white', 'marginBottom': '10px'}),
    dcc.Graph(id='bar-chart'),
    dcc.Interval(
        id='interval-component',
        interval=3*1000,  # 3秒刷新一次
        n_intervals=0
    )
], style={'backgroundColor': '#222', 'padding': '20px'})

@app.callback(
    [Output('bar-chart', 'figure'),
     Output('summary-box', 'children')],
    Input('interval-component', 'n_intervals')
)
def update_bar_chart(n):
    df = fetch_latest_row()
    if df is None or df.empty:
        return go.Figure(), ""
    row = df.iloc[0]
    shanghai_time = pd.to_datetime(row['TIMESTAMP']) + pd.Timedelta(hours=8)
    shanghai_time_str = shanghai_time.strftime('%Y-%m-%d %H:%M:%S')
    y = [
        row['price_range_0'], row['price_range_1'], row['price_range_2'],
        row['price_range_3'], row['price_range_4'], row['price_range_5'],
        row['price_range_6'], row['price_range_7'], row['price_range_8']
    ]
    # 统计上涨、下跌、涨停、跌停
    up_count = int(sum([row['price_range_5'], row['price_range_6'], row['price_range_7'], row['price_range_8']]))
    down_count = int(sum([row['price_range_0'], row['price_range_1'], row['price_range_2'], row['price_range_3']]))
    up_limit = int(row['num_upper_limit'])
    down_limit = int(row['num_lower_limit'])

    summary = [
        html.Span(f"上涨: {up_count}", style={'color': 'red', 'marginRight': '20px'}),
        html.Span(f"涨停: {up_limit}", style={'color': 'red', 'marginRight': '40px'}),
        html.Span(f"下跌: {down_count}", style={'color': 'lime', 'marginRight': '20px'}),
        html.Span(f"跌停: {down_limit}", style={'color': 'lime'})
    ]

    fig = go.Figure(
        data=go.Bar(x=price_range_labels, y=y, marker_color='deepskyblue')
    )
    fig.update_layout(
        title=f"时间: {shanghai_time_str}",
        xaxis_title='涨跌幅区间',
        yaxis_title='股票数量',
        plot_bgcolor='#222',
        paper_bgcolor='#222',
        font_color='white'
    )
    return fig, summary

if __name__ == '__main__':
    app.run(debug=True, port=8052)